In [ ]:
from pathlib import Path
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
out = Path("out")

ckpt_red = pickle.load(open(out / "mc_red.p", "rb"))
df_red = pd.DataFrame(ckpt_red["fit"].T, columns=[str(l) for l in ckpt_red["train_lens"]])
df_red = df_red.melt(var_name="length", value_name="fit")
df_red.insert(0, "model", "reduced")

In [ ]:
out = Path("out")

ckpt_full = pickle.load(open(out / "mc_full.p", "rb"))
#fit_best = np.copy(ckpt_full["fit_bfgs"])
#mask = (ckpt_full["fit_adam_tr"] > ckpt_full["fit_bfgs_tr"]) | (np.isnan(ckpt_full["fit_bfgs_tr"]))
#fit_best[mask] = ckpt_full["fit_adam"][mask]

df_full = pd.DataFrame(ckpt_full["fit_bfgs"].T, columns=[str(l) for l in ckpt_full["train_lens"]])
df_full = df_full.melt(var_name="length", value_name="fit")
df_full.insert(0, "model", "full")
#df_full.dropna(inplace=True)
#df_full.drop(df_full[df_full.fit < 0].index, inplace=True)


df_mc = pd.concat((df_full, df_red), ignore_index=True)

In [ ]:
#fit_best[mask], 
#ckpt_full["fit_bfgs"][mask]

In [ ]:
#ckpt_full["fit_adam"][mask]

In [ ]:
rms_y = 66.69 # 1e5 rmse of y 
fit_lin = 77.17 # fit of a linear baseline obtained with MATLAB's sysid toolbox
rmse_lin = (1 - fit_lin/100.0) * rms_y # rmse of a linear baseline
rmse_lin/1e5

In [ ]:
df_mc["fit"] = np.maximum(df_mc["fit"], 0.0)

df_mc["rmse"] = (1 - df_mc["fit"]/100.0) * rms_y

In [ ]:
#tmp = df_mc[df_mc["fit"] < 1].groupby(["model", "length"]).fit.count()
#tmp["full (bfgs)"]

In [ ]:
model_labels = {
#    "full": "full parameterization",
#    "reduced": "reduced parameterization",
    # Add more mappings as needed
}
model_order = ["full", "reduced"]

plt.figure(figsize=(8, 4))
ax = sns.boxplot(df_mc, x="length", y="fit", hue="model")#, hue_order=["full", "reduced"])
plt.tight_layout()  # This adjusts the plot to ensure the legend fits
plt.xticks(rotation=45)
ax.set_xlabel("Training sequence length (-)")
ax.set_ylabel("Test FIT (%)")
ax.axhline(y=fit_lin, color="black", linestyle="dotted", alpha=0.5, label="LTI 40960 samples")#, linewidth=0.5)
ax.axhline(y=98.8, color="black", linestyle="dashed", alpha=0.5, label="Full-order 40960 samples")#, linewidth=0.5)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.legend(loc='lower right')#, borderaxespad=0)
plt.ylim([-5, 100])
plt.tight_layout()
plt.savefig(Path("fig")/"boxplot.pdf")
#plt.savefig(Path("fig")/"boxplot.png")

In [ ]:
ckpt_full.keys()

In [ ]:
plt.figure()
plt.title("Training time (hours)")
plt.plot(ckpt_full['train_lens'], ckpt_full['train_time']/60/24, "*", label="full (adam + BFGS)")
plt.plot(ckpt_red['train_lens'], ckpt_red['train_time']/60/24, "*", label="reduced")

#plt.plot(df_full_time.length, df_full_time.time_adam/60/24, "*", label="full (adam)")
plt.xlabel("Training sequence length (-)")
plt.ylabel("Time (hours)")
plt.legend();